## Initialitation

In [1]:
from pycqed.init.QuDev_testbench_init import *

Data directory set to: E:\Control software\data


Could not import qutip
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch1_m1_del at 201505256>
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch1_m2_del at 201506096>
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch2_m1_del at 201521696>
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch2_m2_del at 201522424>
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch3_m1_del at 201529832>
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch3_m2_del at 201530560>
`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch4_m1_d

initializing SGS100A's
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/103189, firmware:3.1.19.15-3.20.347.26) in 0.05s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/103188, firmware:3.1.19.15-3.20.347.26) in 0.02s
Connected to: Rohde&Schwarz SGS100A (serial:1416.0505k02/103718, firmware:3.1.19.15-3.20.347.26) in 0.02s
initializing AWG5014


`units` is deprecated for the `Parameter` class, use `unit` instead. <qcodes.instrument.parameter.StandardParameter: ch4_m2_del at 201583752>


Connected to: TEKTRONIX AWG5014C (serial:B010154, firmware:SCPI:99.0 FW:4.6.0.7) in 0.15s
initializing UHFQC
autoDetect selected the device dev2204 for the measurement.
Uploading file to device...
File successfully uploaded
Initialized UHFQC dev2204 in 1.05s
initializing heterodynes
initializing qubit


e:\control software\qcodes\qcodes\plots\pyqtgraph.py:46: UserWarning: windowTitle argument has been changed to window_title. Please update your call to QtPlot
  warnings.warn("windowTitle argument has been changed to window_title. Please update your call to QtPlot")


configuring parameters


Error getting or interpreting *IDN?: ''
Error getting or interpreting *IDN?: ''


Ran initialization in 11.59s


## Find the resonator frequency

In [ ]:
heterodyne.RF_power(-40)
qb2.measure_heterodyne_spectroscopy(freqs=np.linspace(7.19e9,7.191e9,20), close_fig=True)
#qb2.find_resonator_frequency(freqs=np.linspace(8.10e9,8.12e9,50), close_fig=False)
#qb2.find_resonator_frequency(close_fig=False)
#print("f0 = {} Hz".format(qb2.f_RO_resonator()))
#plt.show()

In [ ]:
MA = ma.MeasurementAnalysis()
MA.sweep_points[np.argmin(MA.measured_values[0])]

In [2]:
qb2.f_RO(7190420992.0)
heterodyne.frequency(7190420992.0)

## Calibrate the acquisition delay (for LO_modulated_heterodyne)

In [ ]:
heterodyne.frequency(7.1795e9)
qubit.measure_homodyne_acqusition_delay(delays=[0,100e-9],analyze=True) # this is necessary because of a bug where the first 
                                                                         # datapoint might be garbled
#qubit.find_homodyne_acqusition_delay(close_fig=False)
#print("delay = {} ns".format(qubit.optimal_acquisition_delay()))
#plt.show()

## Qubit spectroscopy

In [ ]:
qb2.spec_pow(-40)
readout_RF.power(-40)
qb2.measure_spectroscopy(freqs=np.linspace(6.023e9, 6.0245e9, 50), pulsed=True)

In [ ]:
QSA = ma.Qubit_Spectroscopy_Analysis(label='qb2',show_guess=False)
QSA.fit_results[0].values['f0']

In [3]:
qb2.f_qubit(6023614771.193965)

## Sweep acquisition delay (for calibration only)

In [ ]:
qubit.prepare_for_pulsed_spec()

spec_pars = qubit.get_spec_pars()
RO_pars = qubit.get_RO_pars()

AWG.clock_freq(1e9)
heterodyne.RO_length(2.2e-6)
heterodyne.nr_averages(1024)
qubit.cw_source.frequency(5.98e9)
qubit.heterodyne.frequency(7.1695e9)
qubit.spec_pow(-50)
qubit.readout_RF.power(-20)
qubit.RO_acq_marker_delay(-5e-6)
qubit.RO_amp()
qubit.cw_source.on()

sq.Pulsed_spec_seq(spec_pars, RO_pars)

qubit.AWG.start()

MC.set_sweep_function(UHFQC.awgs_0_userregs_2)
MC.set_sweep_points(np.arange(0, 1400, 25))
demod_mode = 'single' if qubit.heterodyne.single_sideband_demod() else 'double'
MC.set_detector_function(det.Heterodyne_probe(
    qubit.heterodyne,
    trigger_separation=qubit.heterodyne.trigger_separation(),
    demod_mode=demod_mode))
MC.soft_avg(1)
MC.run(name='pulsed-spec' + qubit.msmt_suffix)

qubit.cw_source.off()

## Rabi experiment
Remember to set the switch on upconversion board 1 to correct position

In [ ]:
qb2.measure_rabi(amps=np.linspace(0.3, 0.45, 31))

In [ ]:
#RabiA = ma.Rabi_Analysis(label='qubit')
RabiPA = ma.Rabi_parabola_analysis(label='qb2')
(RabiPA.fit_res[0].values['x0'] + RabiPA.fit_res[1].values['x0'])/2

In [4]:
qb2.amp180(0.36664373289086)

## Ramsey

In [ ]:
artificial_detuning = 4e6
qb2.measure_ramsey(times=np.linspace(0,4e-6,100), artificial_detuning=artificial_detuning)

In [ ]:
RamA = ma.Ramsey_Analysis(auto=True)

fitted_freq = RamA.fit_res.params['frequency'].value
print('Artificial detuning: {:.2e}'.format(artificial_detuning))
print('Fitted detuning: {:.2e}'.format(fitted_freq))
print('Actual detuning:{:.2e}'.format(fitted_freq-artificial_detuning))
qb2.f_qubit() - fitted_freq + artificial_detuning

In [5]:
qb2.f_qubit(6023619855.1445322)

## Rabi 90 amplitude

In [ ]:
qb2.measure_rabi_amp90(scales=np.linspace(0.4, 0.6, 31))

In [ ]:
Rabi90PA = ma.Rabi_parabola_analysis(label='qb2')
print("mean({}, {}) = {}".format(
    Rabi90PA.fit_res[0].values['x0'], 
    Rabi90PA.fit_res[1].values['x0'], 
    (Rabi90PA.fit_res[0].values['x0'] + Rabi90PA.fit_res[1].values['x0'])/2))

In [6]:
qb2.amp90_scale(0.5008861843621694)

## T1 measurement

In [ ]:
qb2.measure_T1(times=np.linspace(0,40e-6,100))

In [9]:
T1A = ma.T1_Analysis(auto=True, close_fig=True)
T1 = T1A.T1
T1

## Spin-echo experiment

In [ ]:
artificial_detuning = 4e6 #Hz
qb2.measure_echo(times=np.linspace(0,4e-6,100), artificial_detuning=artificial_detuning)

In [ ]:
SEA = ma.Ramsey_Analysis(auto=True, close_fig=True, label='Echo')
print("T2 = {}".format(SEA.fit_res.params['tau'].value))

## AllXY experiment
Measures different pairs of I, X180, Y180, X90 and Y90 pulses.

In [ ]:
qb2.measure_allxy()

In [ ]:
AllXYA = ma.AllXY_Analysis()

## Randomized Benchmarking

In [7]:
nr_cliffords=[2,4,8,16]
qb2.measure_randomized_benchmarking(nr_cliffords)

Uploading file to device...
File successfully uploaded
Starting measurement: RB_50seeds_qb2
Sweep function: Randomized_Benchmarking
Detector function: UHFQC_integrated_average


No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using default config from station
No sequencer config detected, using defa

Programming 'RandomizedBenchmarking_sequence' (600 element(s)) 	  finished in 28.22 seconds.
 100% completed 	elapsed time: 88.2s 	time left: 0.0s


In [10]:
RBA = ma.RandomizedBenchmarking_Analysis(T1=T1, pulse_delay=qb2.pulse_delay())

## Butterfly experiment (still needs testing)

In [ ]:
from pycqed.measurement import awg_sweep_functions as awg_swf
reload(sq)
sq.station = station

qubit.int_log_det = det.UHFQC_integration_logging_det(
    UHFQC=UHFQC, AWG=AWG, channels=[
        qubit.RO_acq_weight_function_I(),
        qubit.RO_acq_weight_function_Q()],
    integration_length=qubit.RO_acq_integration_length(),
    nr_shots=4092)

def measure_butterfly(self, return_detector=False,
                      MC=None,
                      initialize=False,
                      post_msmt_delay=2e-6, case=True):
    self.prepare_for_timedomain()
    if MC is None:
        MC = self.MC
    MC.set_sweep_function(awg_swf.Butterfly(
        pulse_pars=self.get_drive_pars(), RO_pars=self.get_RO_pars(),
        initialize=initialize, post_msmt_delay=post_msmt_delay))
    MC.set_detector_function(self.int_log_det)
    MC.run('Butterfly{}initialize_{}'.format(self.msmt_suffix, initialize))

measure_butterfly(qubit)

In [ ]:
reload(ma)
# first perform SSRO analysis to extract the optimal rotation angle
# theta
a = ma.SSRO_discrimination_analysis(
    label='Butterfly',
    current_threshold=None,
    close_fig=False,
    plot_2D_histograms=True)

# the, run it a second time to determin the optimum threshold along the
# rotated I axis
b = ma.SSRO_discrimination_analysis(
    label='Butterfly',
    current_threshold=None,
    close_fig=False,
    plot_2D_histograms=True, theta_in=-a.theta)

c = ma.butterfly_analysis(
    close_main_fig=False, initialize=initialize,
    theta_in=-a.theta,
    threshold=b.opt_I_threshold, digitize=True, case=True)

In [ ]:
plt.show()

In [ ]:
c.butterfly_coeffs

In [ ]:
plt.plot(a.measured_values[1])
plt.show()

## temp

In [ ]:
#UHFQC.quex_rl_length(2)
AWG.start()
UHFQC.awgs_0_enable(1)

# why do we need this?
try:
    UHFQC.awgs_0_enable()
except:
    UHFQC.awgs_0_enable()

while UHFQC.awgs_0_enable() == 1:
    time.sleep(0.01)
data = ['', '']
data[0] = UHFQC.quex_rl_data_0()[0]['vector']
data[1] = UHFQC.quex_rl_data_1()[0]['vector']
AWG.stop()
data
#data[0][0]**2 + data[1][0]**2

In [ ]:
reload(ZI_UHFQC)
UHFQC.single_acquisition_initialize()
data = UHFQC.single_acquisition(2)
UHFQC.single_acquisition_finalize()
data

In [ ]:
qubit.cw_source.power(-40)
qubit.spec_pulse_length(20e-6) #s
qubit.spec_pulse_depletion_time(10e-6) #s
qubit.RO_pulse_length(2.2e-6) #s
qubit.RO_acq_marker_delay(0) #s
qubit.RO_pulse_delay(500e-9) #s
readout_RF.pulsemod_state('On')

In [ ]:
qubit.measure_spectroscopy(freqs=np.linspace(5.8e9, 6.2e9, 50), pulsed=True)

In [ ]:
qubit.readout_RF.close()
qubit.readout_LO.close()

In [ ]:
station.components.keys()

In [ ]:
AWG.clock_freq(1e9)

In [ ]:
readout_LO.power()

In [ ]:
heterodyne.nr_averages()

In [ ]:
qubit.RO_pulse_power()

In [ ]:
readout_RF.pulsemod_state("On")

In [ ]:
for instr in station.components.values():
    instr.close()